In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

In [2]:
frame_num = 50
file_path = r'C:\Users\zhang\Desktop\share\18knpt_1000p_out.xyz'
creat_new_frame = 1

In [3]:
#读取文本函数，返回直接d选帧
def get_data(path,frame_num):
    with open(path) as f:
        str_data = f.read()
    data = str_data.split()
    ###选帧#########
    #此处写个程序选帧
    atoms_num = int(data[0])
    frame_len = atoms_num*4+2
    data_len =len(data)
    max_frame = data_len // frame_len
    
    temp = []
    temp = data[frame_len*frame_num:frame_len*(frame_num+1)]

    #暂时先自行处理了表头
    temp = temp[2:]
    frame_list = []
    for i in range(atoms_num):
        frame_list.append([temp[i*4+0],float(temp[i*4+1]),float(temp[i*4+2]),float(temp[i*4+3])])
    
    return frame_list

# 主函数

In [4]:
new_name = r'C:\Users\zhang\Desktop\frame_ana_cry.xyz'
if(creat_new_frame==1):
    new_name = "C:\\Users\zhang\\Desktop\\"
    new_name += str(frame_num) + 'frame.xyz'

data = get_data(file_path,frame_num)

In [5]:
data[0]

['O', 24.979952, 5.559612, 24.81585]

In [6]:
data[1]

['O', 37.970052, 36.308752, 13.46295]

In [7]:
# 先设置最大值xyz的最大值 由于其为npt，所以其xyz边界是变化的，那么最大的就动态获取
max_xyz = 0
dic_ele = {}
ba_list = []
all_atoms = 0
for i in data:
    all_atoms+=1
    if i[1]>max_xyz:
        max_xyz = i[1]
    if i[2]>max_xyz:
        max_xyz = i[2]
    if i[3]>max_xyz:
        max_xyz = i[3]
        
    # 统计元素种类及个数
    if i[0] not in dic_ele:
        dic_ele[i[0]] = 0
    else:
        dic_ele[i[0]]+=1
    # 得到ba列表
    if i[0]=='Ba':
        ba_list.append(i.copy())
print(max_xyz)    
print(dic_ele)

57.32145
{'O': 6333, 'Si': 1666, 'Al': 1999, 'F': 1333, 'Ba': 666}


In [8]:
ba_num = 0
si_num = 0
al_num = 0
f_num = 0
o_num = 0
for i in data:
    if i[0]=='Ba':
        ba_num+=1
    elif i[0]=='Si':
            si_num+=1   
    elif i[0]=='Al':
        al_num+=1
    elif i[0]=='F':
        f_num+=1
    elif i[0]=='O':
        o_num+=1
    else:
        print(i)
print(ba_num,si_num,al_num,f_num,o_num)

667 1667 2000 1334 6334


In [9]:
len(data)

12002

In [10]:
# 定义距离函数
def cal_distance(x1,y1,z1,x2,y2,z2):
    dis = ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(1/2)
    if dis<1:
        return 10
    return dis

In [11]:
# 定义坐标转换函数
def tran_coordinate(x1,y1,z1,x2,y2,z2):
    pass

1.如果一个Ba原子的第一近邻大多数都为F
2.Ba原子的中程形成体大多数都是Ba 而Al和Si很少

In [12]:
# 定义分析函数
def ana_separation(n1,x1,y1,z1,n2,x2,y2,z2):
    pass

In [13]:
for ba_i in ba_list:
    count_ba_ba = 0
    count_al_si = 0
    count_o = 0
    count_f = 0
    
    for j in data:
        # 如果是同一原子则跳过 这个在计算距离的时候判断就行了  不然计算太多遍了
        # if (i[1]-j[1]<1e-4 and i[2]-j[2]<1e-4 and i[3]-j[3]
        if j[0]=='Ba' or j[0]=='Cs':
            if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 6.0:
                count_ba_ba+=1
        if j[0]=='Al' or j[0]=='Si':
            if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 6.0:
                count_al_si+=1
        if j[0]=='O' :
            if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 3.5:
                count_o+=1
        if j[0]=='F' :
            if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 3.5:
                count_f+=1
        
        #综合判断
    if(count_al_si<=count_ba_ba and count_o <= count_f):
        ba_i[0]='Cs'

In [14]:
num = 0
for i in ba_list:
    if i[0]=='Cs':
        num+=1

In [15]:
print(dic_ele)
print("separation Ba:",num)

{'O': 6333, 'Si': 1666, 'Al': 1999, 'F': 1333, 'Ba': 666}
separation Ba: 108


# 保存文件

#看文件是否存在 存在也删了 都要重建
if not os.path.exists(new_name):
    os.system(r'touch {}'.format(new_name))
else:
    os.remove(new_name)
    os.system(r'touch {}'.format(new_name))

In [16]:
#看文件是否存在 存在也删了 都要重建
if not os.path.exists(new_name):
    os.system(r'touch {}'.format(new_name))
else:
    os.remove(new_name)
    os.system(r'touch {}'.format(new_name))

In [17]:
count = 1
count_ba = 0
#先把文件头写上去
with open(new_name,'a') as file:
        file.write("{} \n".format(all_atoms))
        file.write("{} \n".format("Atoms"))
#在两个循环里分别写入原子
with open(new_name,'a') as file:
    for i in data:
        if i[0]!='Ba':
            count+=1
            file.write("{}    {}    {}    {}\n".format(i[0],i[1],i[2],i[3]))
    for j in ba_list:
        count_ba +=1
        count+=1
        file.write("{}    {}    {}    {}\n".format(j[0],j[1],j[2],j[3]))
        if j[0]=='Cs':
            print(j)

print(count,count_ba)

['Cs', 26.903252, 1.98128, 40.44975]
['Cs', 7.346942, 12.036052, 17.25135]
['Cs', 5.884362, 18.878552, 19.23575]
['Cs', 13.237352, 23.395652, 5.75637]
['Cs', 2.225521, 33.179852, 4.38447]
['Cs', 12.474052, 26.126552, 2.18953]
['Cs', 16.953152, 30.081352, 3.7602]
['Cs', 3.479232, 1.105017, 7.55694]
['Cs', 0.939512, 54.172752, 30.65365]
['Cs', 54.949052, 51.806752, 29.53465]
['Cs', 18.156652, 26.573952, 53.80425]
['Cs', 8.001832, 2.930012, 14.07755]
['Cs', 2.289784, 10.596902, 16.62175]
['Cs', 15.567152, 28.968452, 0.28568]
['Cs', 8.648382, 6.700782, 9.52353]
['Cs', 9.993082, 25.768552, 6.56165]
['Cs', 54.395152, 48.653852, 35.65795]
['Cs', 8.386562, 15.407652, 21.82435]
['Cs', 4.093952, 29.328552, 56.36975]
['Cs', 6.823572, 30.905052, 5.83151]
['Cs', 8.855582, 16.378052, 17.66165]
['Cs', 18.369352, 2.26741, 43.11915]
['Cs', 4.135092, 2.815662, 14.28075]
['Cs', 5.512212, 26.835452, 7.3577]
['Cs', 10.827682, 11.759852, 12.95485]
['Cs', 16.026352, 4.517192, 41.37485]
['Cs', 29.388952, 9.21

In [18]:
len(data)

12002

In [19]:
a = 0
for i in data:
    if i[0]=='Cs':
        a+=1
a

0

In [20]:
xx = 'wqeqw'
yy = '41241'
with open(new_name,'a') as file:
        file.write(xx+yy)
        file.write("{} \n".format("Atoms"))